# ------------------------GDAL---------------------------

## 矢量数据

In [8]:
#读取shp文件 获得要素数量
import os
from osgeo import ogr

daShapefile = r"fiona_test.shp"

driver = ogr.GetDriverByName('ESRI Shapefile')

dataSource = driver.Open(daShapefile, 0) # 0 means read-only. 1 means writeable.

# Check to see if shapefile is found.
if dataSource is None:
    print('Could not open %s'%(daShapefile))
else:
    print ('Opened %s' % (daShapefile))
    layer = dataSource.GetLayer()
    featureCount = layer.GetFeatureCount()
    print("Number of features in %s: %d" % (os.path.basename(daShapefile),featureCount))

Opened fiona_test.shp
Number of features in fiona_test.shp: 2


In [2]:
#获得数据库中所有图层
from osgeo import ogr

databaseServer = "192.168.65.1"
databaseName = "geohey"
databaseUser = "jingcb"
databasePW = "290504"


connString = "PG: host=%s dbname=%s user=%s password=%s" %(databaseServer,databaseName,databaseUser,databasePW)

conn = ogr.Open(connString)

layerList = []
for i in conn:
    daLayer = i.GetName()
    if not daLayer in layerList:
        layerList.append(daLayer)

layerList.sort()

for j in layerList:
    print(j)

# Close connection
conn = None

base.administration_township
bigdata
fangtest
fangtest_beijing_sub
fangtest_point
poi_dis
town
world_country_point
world_country_polygon


In [3]:
#获得 数据库图层的要素数量
import sys
def GetPGLayer( lyr_name ):
    conn = ogr.Open(connString)

    lyr = conn.GetLayer( lyr_name )
    if lyr is None:
        print >> sys.stderr, '[ ERROR ]: layer name = "%s" could not be found in database "%s"' % ( lyr_name, databaseName )
        sys.exit( 1 )

    featureCount = lyr.GetFeatureCount()
    print ("Number of features in %s: %d" % ( lyr_name , featureCount ))

    # Close connection
    conn = None

In [4]:
GetPGLayer('fangtest')

Number of features in fangtest: 2305


In [5]:
#获取要素的geometry
from osgeo import ogr
import os

shapefile = "fiona_test.shp"
driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(shapefile, 0)
layer = dataSource.GetLayer()

for feature in layer:
    geom = feature.GetGeometryRef()
    print(geom.Centroid().ExportToWkt())

POINT (-87.63 41.88)
POINT (-94.584 39.101)


In [7]:
#属性过滤
from osgeo import ogr
import os

shapefile = "fiona_test.shp"
driver = ogr.GetDriverByName("ESRI Shapefile")
dataSource = driver.Open(shapefile, 0)
layer = dataSource.GetLayer()

layer.SetAttributeFilter("name = 'Chicago'")

for feature in layer:
    print (feature.GetField("name"))

Chicago


In [8]:
#获取属性名
from osgeo import ogr

daShapefile = r"fiona_test.shp"

dataSource = ogr.Open(daShapefile)
daLayer = dataSource.GetLayer(0)
layerDefinition = daLayer.GetLayerDefn()


for i in range(layerDefinition.GetFieldCount()):
    print(layerDefinition.GetFieldDefn(i).GetName())

name


## 栅格影像

In [2]:
#打开数据，获取影像元数据
from osgeo import gdal
gtif = gdal.Open( "images/input.tif" )
print(gtif.GetMetadata())

{'AREA_OR_POINT': 'Point'}


In [6]:
# 获取波段信息
srcband = gtif.GetRasterBand(1)

In [10]:
# 统计信息
srcband.GetStatistics( True, True )   #最小像素值， 最大， 平均，标准偏差

[8896.0, 33262.0, 10320.252106409176, 882.4653425033526]

In [13]:
print("[ NO DATA VALUE ] = ", srcband.GetNoDataValue())
print( "[ SCALE ] = ", srcband.GetScale())
print( "[ UNIT TYPE ] = ", srcband.GetUnitType())

[ NO DATA VALUE ] =  0.0
[ SCALE ] =  1.0
[ UNIT TYPE ] =  


In [ ]:
#某一个波段的矢栅转换
from osgeo import gdal, ogr
import sys
# this allows GDAL to throw Python Exceptions
gdal.UseExceptions()

#
#  get raster datasource
#
src_ds = gdal.Open( "images/input.tif" )
if src_ds is None:
    print('Unable to open %s' % src_filename)
    sys.exit(1)

try:
    srcband = src_ds.GetRasterBand(1)
except RuntimeError as e:
    # for example, try GetRasterBand(10)
    print('Band ( %i ) not found' % 1)
    print (e)
    sys.exit(1)

#
#  create output datasource
#
dst_layername = "POLYGONIZED_STUFF"
drv = ogr.GetDriverByName("ESRI Shapefile")
dst_ds = drv.CreateDataSource( dst_layername + ".shp" )
dst_layer = dst_ds.CreateLayer(dst_layername, srs = None )

gdal.Polygonize( srcband, None, dst_layer, -1, [], callback=None )

# -------------------------rasterio----------------------------

In [1]:
import rasterio

In [4]:
#读取tiff文件
dataset = rasterio.open('images/input.tif')

In [5]:
dataset.name

'images/input.tif'

In [6]:
dataset.mode

'r'

In [7]:
dataset.closed

False

In [8]:
dataset.count

1

In [9]:
dataset.width

7751

In [10]:
dataset.height

7881

In [11]:
dataset.bounds

BoundingBox(left=358185.0, bottom=4346085.0, right=590715.0, top=4582515.0)

In [13]:
#仿射变换矩阵
dataset.transform

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


[358185.0, 30.0, 0.0, 4582515.0, 0.0, -30.0]

In [66]:
#利用变换矩阵求取某一像素点的地理坐标
[1,0,0]@(np.array(dataset.transform).reshape(2,3).T)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


array([  358185.,  4582515.])

In [67]:
[1,7,7]@(np.array(dataset.transform).reshape(2,3).T)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2910: FutureWarning: The value of this property will change in version 1.0. Please see https://github.com/mapbox/rasterio/issues/86 for details.
  exec(code_obj, self.user_global_ns, self.user_ns)


array([  358395.,  4582305.])

In [77]:
#新版本
dataset.xy(7, 7)

AttributeError: 'rasterio._io.RasterReader' object has no attribute 'xy'

In [69]:
#坐标系
dataset.crs

CRS({'init': 'epsg:32650'})

In [70]:
#波段
dataset.read(1)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint16)

##### read()方法返回的是numpy数组

In [76]:
#行列号索引
dataset.read(1)[dataset.height // 2, dataset.width // 2]

10716

In [75]:
#空间索引
x, y = (dataset.bounds.left + 100000, dataset.bounds.top - 50000)
row, col = dataset.index(x, y)
row, col

(1666, 3333)